## Purchases

In [1]:
# Import dependency and file, load file into dataframe
import pandas as pd
file_input = "Resources/purchase_data.csv"
purchase_data = pd.read_csv(file_input)
purchase_data.head(10)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
5,5,Yalae81,22,Male,81,Dreamkiss,3.61
6,6,Itheria73,36,Male,169,"Interrogator, Blood Blade of the Queen",2.18
7,7,Iskjaskst81,20,Male,162,Abyssal Shard,2.67
8,8,Undjask33,22,Male,21,Souleater,1.10
9,9,Chanosian48,35,Other / Non-Disclosed,136,Ghastly Adamantite Protector,3.58


## Player Count

In [2]:
player_total = purchase_data["SN"].nunique()
print(f'{player_total} players')

576 players


## Overall Purchasing Analysis

In [3]:
analyzed_purchases = {
  "unique_items": purchase_data["Item ID"].nunique(), #Count unique items
  "average_price": round(purchase_data["Price"].mean(),2), #Calculate average price and round to two decimals
  "total_purchases": purchase_data["Purchase ID"].nunique(), #Calculable total of orders
  "total_revenue": round(purchase_data["Price"].sum(),2) #Calculate total revenue and round to two decimals
}

#Create dataframe of analyzed purchases
analyzed_purchases_df=pd.DataFrame({"Number of Unique Items":[analyzed_purchases["unique_items"]],"Average Purchase Price":[analyzed_purchases["average_price"]],"Total Number of Purchases":[analyzed_purchases["total_purchases"]], "Total Revenue":[analyzed_purchases["total_revenue"]]})

analyzed_purchases_df

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,3.05,780,2379.77


## Gender Demographics

In [4]:
#Calculate gender totals and percentages
gender_demographics_df=purchase_data.groupby(["Gender"]).nunique()
gender_demographics_df=gender_demographics_df["SN"]
gender_demographics_df_total=gender_demographics_df.sum()
gender_demographics_percentage=(gender_demographics_df/gender_demographics_df_total)

# Clean calculated gender totals and percentages to display
analyzed_gender_demographics=pd.merge(gender_demographics_df, gender_demographics_percentage, on='Gender')
analyzed_gender_demographics=analyzed_gender_demographics.rename(columns={"SN_x":"Total Count","SN_y":"Percentage of Players"})
analyzed_gender_demographics["Percentage of Players"]=analyzed_gender_demographics["Percentage of Players"].map("{:.2%}".format)
analyzed_gender_demographics=analyzed_gender_demographics.sort_values("Total Count", ascending=False)
analyzed_gender_demographics

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis By Gender

In [5]:
# Calculate gender purchase counts and create tables
demo_purch=purchase_data.groupby(["Gender"]).nunique()["Purchase ID"]
demo_avgprc=purchase_data.groupby(["Gender"]).mean()["Price"]
demo_totpur=purchase_data.groupby(["Gender"]).sum()["Price"]
tot_pur_per_person=purchase_data.groupby(["SN","Gender"]).sum()
avg_tot_per_per_demo=tot_pur_per_person.groupby("Gender").mean()
avg_tot_per_per_demo=avg_tot_per_per_demo["Price"]

# Merge tables and load into dataframe
demo_sum_purch=pd.merge(demo_purch, demo_avgprc, on='Gender')
demo_sum_purch=pd.merge(demo_sum_purch, demo_totpur, on='Gender')
demo_sum_purch=pd.merge(demo_sum_purch, avg_tot_per_per_demo, on='Gender')

# Clean calculated gender purchase totals to display
demo_sum_purch=demo_sum_purch.rename(columns={"Purchase ID":"Purchase Count","Price_x":"Average Purchase Price", "Price_y":"Total Purchase Value","Price":"Avg Total Purchase per Person"})
demo_sum_purch["Average Purchase Price"]=demo_sum_purch["Average Purchase Price"].map("${:.2f}".format)
demo_sum_purch["Total Purchase Value"]=demo_sum_purch["Total Purchase Value"].map("${:.2f}".format)
demo_sum_purch["Avg Total Purchase per Person"]=demo_sum_purch["Avg Total Purchase per Person"].map("${:.2f}".format)
demo_sum_purch=demo_sum_purch.sort_values("Purchase Count", ascending=False)
demo_sum_purch


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Male,652,$3.02,$1967.64,$4.07
Female,113,$3.20,$361.94,$4.47
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

In [6]:
# Generate bins and group names and add to dataframe
bins = [0, 9, 14, 19, 24, 29, 34,39, 150]
bin_names=["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
purchase_data["Groups"]=pd.cut(purchase_data["Age"], bins, labels=bin_names, include_lowest=True)

#Calculate total amount of people and percentage of groups
total_people=purchase_data["SN"].nunique()
age_demographics=purchase_data.groupby(["Groups"]).nunique()["SN"]
age_demographics_percentage=(age_demographics/total_people)
age_demographics_percentage

# Merge demographics and percentages together and clean results to display
output=pd.merge(age_demographics, age_demographics_percentage.map("{:.2%}".format), on='Groups')
output=output.rename(columns={"SN_x":"Total Count","SN_y":"Percentage of Players"})
output

,Total Count,Percentage of Players
Groups,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis By Age

In [7]:
purchase_count=purchase_data.groupby(["Groups"]).nunique()["Purchase ID"] #Calculate purchase count per group
overall_average_price=purchase_data.groupby(["Groups"]).mean()["Price"] #Calculate average price per group
overall_total_price=purchase_data.groupby(["Groups"]).sum()["Price"] #Calculate total price per group
total_group_purchase=purchase_data.groupby(["SN","Groups"]).sum() #Calculate total price per group per person
avgerage_group_total=total_group_purchase.groupby("Groups").mean()["Price"] #Calculate average total per person per group

# Merge data and clean results to display
purchasing_by_age=pd.merge(purchase_count, overall_average_price, on='Groups')
purchasing_by_age=pd.merge(purchasing_by_age, overall_total_price, on='Groups')
purchasing_by_age=pd.merge(purchasing_by_age, avgerage_group_total, on='Groups')
purchasing_by_age=purchasing_by_age.rename(columns={"Purchase ID":"Purchase Count","Price_x":"Average Purchase Price", "Price_y":"Total Purchase Value","Price":"Avg Total Purchase per Person"})
purchasing_by_age["Average Purchase Price"]=purchasing_by_age["Average Purchase Price"].map("${:.2f}".format)
purchasing_by_age["Total Purchase Value"]=purchasing_by_age["Total Purchase Value"].map("${:.2f}".format)
purchasing_by_age["Avg Total Purchase per Person"]=purchasing_by_age["Avg Total Purchase per Person"].map("${:.2f}".format)
purchasing_by_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Groups,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

In [8]:
# Group sn purchases together by sn and purchase id
sn_purchases=purchase_data.groupby(["SN"]).nunique().sort_values("Purchase ID", ascending=False)["Purchase ID"]

sn_purchases_mean=purchase_data.groupby(["SN"]).mean()["Price"] #Calculate average purchase price by SN
sn_purchases_total=purchase_data.groupby(["SN"]).sum()["Price"] #Calculate total purchase price by SN

# Merge dataframes together, clean results, and display 5 with highest total purchase cost
sn_purchases_descending=pd.merge(sn_purchases, round(sn_purchases_mean,2), on='SN')
sn_purchases_descending=pd.merge(sn_purchases_descending, sn_purchases_total, on='SN')
sn_purchases_descending=sn_purchases_descending.rename(columns={"Purchase ID":"Purchase Count","Price_x":"Average Purchase Price", "Price_y":"Total Purchase Value"})
sn_purchases_descending=sn_purchases_descending.sort_values("Total Purchase Value", ascending=False).head(5)
sn_purchases_descending

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.79,18.96
Idastidru52,4,3.86,15.45
Chamjask73,3,4.61,13.83
Iral74,4,3.40,13.62
Iskadarya95,3,4.37,13.10


## Most Popular Items

In [9]:
# Calculate purchase count by item id and purchase id then sort it in descending order
sku_purchases=purchase_data.groupby(["Item ID"]).nunique().sort_values("Purchase ID", ascending=False)["Purchase ID"]
sku_name=purchase_data.groupby(["Item Name","Item ID"]).count()
sku_name=sku_name.reset_index()
sku_name=sku_name[["Item Name","Item ID"]]

sku_mean=purchase_data.groupby(["Item ID"]).mean()["Price"] #Calculate average purchase price
sku_total=purchase_data.groupby(["Item ID"]).sum()["Price"] #Calculate total purchase price

# Merge the data, clean and sort it for display
sku_purchases=pd.merge(sku_purchases, sku_name, on='Item ID')
sku_purchases_descending=pd.merge(sku_purchases, round(sku_mean,2), on='Item ID')
sku_purchases_descending=pd.merge(sku_purchases_descending, sku_total, on='Item ID')
sku_purchases_descending=sku_purchases_descending.rename(columns={"Purchase ID":"Purchase Count","Price_x":"Average Purchase Price", "Price_y":"Total Purchase Value"})
sku_purchases_descending=sku_purchases_descending.sort_values("Purchase Count", ascending=False)
sku_purchases_descending.set_index(['Item ID','Item Name'], inplace=True)
sku_purchases_descending.head(5)


,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,4.61,59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
145,Fiery Glass Crusader,9,4.58,41.22
132,Persuasion,9,3.22,28.99
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [10]:
most_profitable=sku_purchases_descending.sort_values("Total Purchase Value", ascending=False)
most_profitable

,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,4.61,59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
103,Singed Scalpel,8,4.35,34.80
...,...,...,...,...
28,"Flux, Destroyer of Due Diligence",2,1.06,2.12
125,Whistling Mithril Warblade,2,1.00,2.00
126,Exiled Mithril Longsword,1,2.00,2.00


# Three Observable trends from all of this analysis

1. There is a very pronounced bell curve trend in the purchase of games across ages.
2. While there is not quite enough data on people reporting neither male or female gender to say conclusivly, it appears as though other/non-disclosing gendered individuals spend the most on average.
3. The more profitable games are, the more pronounced variance in profit there is.
